# Agent & Rag setup - Howest Student Exercises

Welcome to this hands-on exercise! You'll learn how to:
1. 🤖 Use a Large Language Model (LLM) to generate schedules
2. 📅 Parse and extract schedule information
3. 🔗 Integrate with Google Calendar API
4. ⚡ Create an automated scheduling system using MCP (Model Context Protocol) function calling

## Learning Objectives
By the end of this exercise, you will be able to:
- Set up and use a simple LLM API
- Parse natural language responses into structured data
- Configure Google Calendar API authentication
- Create calendar events programmatically
- Build a complete MCP function calling workflow

## Prerequisites
- Basic Python knowledge
- Google account for Calendar API setup
- VS Code with Python extension installed

Let's get started! 🚀

## Section 1: Setup and Imports

First, let's install and import all the necessary libraries. Run the cell below to install required packages.
The variables are initialized in the config file. Each one of you gets an API key and an Azure Openai Endpoint. 

In [9]:
# Install required packages (run this first!)
# Uncomment the line below if you need to install packages
# !pip install requests google-api-python-client google-auth-httplib2 google-auth-oauthlib

print("📦 Installing packages... (this may take a moment)")
print("✅ Installation complete! Now importing libraries...")

# Core Python libraries
import json
import datetime
import re
import os
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Get Google API credentials from environment variables
Client_id = os.getenv("GOOGLE_CLIENT_ID")
client_secret = os.getenv("GOOGLE_CLIENT_SECRET")

# Here the config file is initialized 
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_deployment_name = os.getenv("OPENAI_DEPLOYMENT_NAME")
openai_version_name = os.getenv("OPENAI_VERSION_NAME")

from pathlib import Path

# HTTP requests for API calls
import requests

# Google Calendar API libraries
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle

print("✅ All libraries imported successfully!")
print("🎯 Ready to start the exercises!")

📦 Installing packages... (this may take a moment)
✅ Installation complete! Now importing libraries...
✅ All libraries imported successfully!
🎯 Ready to start the exercises!


## Exercise 1: Simple LLM Setup 🤖

For this exercise, we'll create a simple function to interact with an LLM. We'll use Azure OpenAI. 

### 📝 Task 1.1: Configure Your API Credentials

**Important**: Replace the placeholder values below with your actual API credentials. 

**For Azure OpenAI users:**
- Get your endpoint and API key from Azure Portal
- Find your deployment name

**For OpenAI users:**
- Get your API key from https://platform.openai.com/

In [2]:
# 🔧 Configuration 
print(azure_openai_endpoint)
print(azure_openai_api_key)
print(openai_deployment_name)
print(openai_version_name)

# 🔍 Choose your LLM provider
print("🔧 Configuration loaded!")

# ⚠️ Security reminder: Never commit API keys to version control!

https://pdpjulie-openai.openai.azure.com/
8rDiuHqogESPRis0AyGuY1WSAqlIpePYfpG5dzTYfHzxFE7SHol3JQQJ99BHACYeBjFXJ3w3AAABACOG3HNO
gpt-4o
2025-03-01-preview
🔧 Configuration loaded!


## Basic LLM Call Template 🤖

Here's a very basic template for making LLM calls. This template shows the essential components you need:

### 1. Basic Structure
```python
def simple_llm_call(prompt, api_endpoint = None, api_key = None, openai_deployment_name = None, openai_version_name = None):
    # Configuration
    api_url = f"{api_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"  # or "api-key": api_key for Azure
    }
    
    # Payload
    payload = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 1000
    }
    
    # Make API call
    response = requests.post(api_url, headers=headers, json=payload)
    
    # Extract response
    result = response.json()
    return result["choices"][0]["message"]["content"]
```

### 2. Key Components:
- **Endpoint**: The API URL where you send requests
- **Headers**: Authentication and content type
- **Payload**: Your prompt and parameters
- **Response Parsing**: Extract the actual text response

### 3. Common Parameters:
- `temperature`: Controls randomness (0.0 = deterministic, 1.0 = creative)
- `max_tokens`: Maximum length of response
- `messages`: Array of conversation messages with roles (system, user, assistant)

This template works for most LLM APIs with minor adjustments!

### 📝 Task 1.2: Create Simple LLM Function

Now let's create a simple function to communicate with the LLM. This function will be the foundation for our schedule generation.
There is only a user prompt (no structured output)

We will mostly use Api requests to Azure to use the LLM 

In [3]:
def simple_llm_call(user_prompt, azure_openai_endpoint, azure_openai_api_key, openai_deployment_name, openai_version_name):
    """
    Simple function to call an LLM with a prompt
    
    Args:
        prompt (str): The prompt to send to the LLM
        max_tokens (int): Maximum tokens in response
    
    Returns:
        str: The LLM's response
    """
    

    # Azure OpenAI API call
    api_url = f"{azure_openai_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
        
    headers = {
            "Content-Type": "application/json",
            "api-key": azure_openai_api_key
        }
        
    payload = {
            "messages": [
                {"role": "user", "content": user_prompt}
            ],
            "temperature": 0.7
        }
    # Make the API call
    response = requests.post(  #POST REQUEST
        api_url,
        headers=headers,
        json=payload,
        timeout=120  # Longer timeout for image processing
        )
    # Parse the response
    response_data = response.json()
    content = response_data["choices"][0]["message"]["content"]
    return content
  
        
# Test the function
print("🧪 Testing LLM function...")
test_response = simple_llm_call("Hello! Can you respond with 'Hello World' to test the connection?", azure_openai_endpoint, azure_openai_api_key, openai_deployment_name, openai_version_name)
print(f"🤖 LLM Response: {test_response}")

🧪 Testing LLM function...
🤖 LLM Response: Hello World


## Exercise 2: Generate Time Schedule with LLM 📅

Now let's ADAPT & USE our LLM to generate a daily schedule! We'll create a prompt that asks the LLM to generate a structured schedule.

### 📝 Task 2.1: Craft an Effective Schedule Prompt

The key to good LLM responses is writing clear, specific prompts. Let's create a prompt that generates a schedule in a consistent format. Using a user prompt, a system prompt & variables. 
Incorporate also few shot prompting. 

In [4]:
schedule_system_prompt = """
You are an expert time management assistant. 
Your task is to generate a realistic and well-structured daily schedule for a student based on the given date and list of tasks.

Return the schedule as a JSON structure with the following fields: 
- date: The date for the schedule (e.g., "2024-10-01")
- start_time: The start time for the task (e.g., "09:00")
- end_time: The end time for the task (e.g., "10:00")
- task: one of the tasks from the provided list
- location: location of the task (e.g., "Office", "Home", "Library")
- type: type of task (e.g., "Study", "Meeting", "Break", "Exercise")

Requirements:
1. Use 24-hour time format (HH:MM)
2. Schedule time slots without overlap, covering the full day from 08:00 to 22:00
3. Include appropriate breaks (e.g., lunch, short rests)
4. Assign suitable locations for each task
5. Categorize each task with a proper type
6. Make realistic and balanced time allocations

Format the response strictly as JSON, following this structure:

{
    "{date}" : {
        "schedule": [
            {
                "start_time": "09:00",
                "end_time": "10:30",
                "task": "Study Session",
                "location": "Library",
                "type": "Study"
            },
            {
                "start_time": "14:00", 
                "end_time": "15:00",
                "task": "Team Meeting",
                "location": "Conference Room A",
                "type": "Meeting"
            }
        ]
    }
}

Only respond with valid JSON — do not include any explanations or text outside the JSON structure.
"""


schedule_user_prompt = """Generate a daily schedule for the date {date} based on the following tasks: {tasks}."""

## Advanced LLM Call Template with Variables 🚀

Here's a more sophisticated template that includes system prompts, user prompts with variables, and advanced configuration options:

### 1. Advanced Structure with System & User Prompts
```python
def advanced_llm_call(user_input, system_prompt=None, variables=None, api_endpoint = None, api_key = None, openai_deployment_name= None, openai_version_name = None):
    # Configuration
    api_url = f"{api_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
    
    # Default system prompt if none provided
    if system_prompt is None:
        system_prompt = "You are a helpful AI assistant."
    
    # Format user input with variables if provided
    # If variables : 
    formatted_user_input = user_input.format(**variables)
    # If there are no variables 
    # formatted_user_input = user_input

    # Headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"  # OpenAI style
    }
    
    # Advanced payload with system and user messages
    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": formatted_user_input}
        ],
        "temperature": 0.1,      # Low for consistent results
        "max_tokens": 4000,      # Longer responses
        "top_p": 0.9,           # Nucleus sampling
        "frequency_penalty": 0,  # Reduce repetition
        "presence_penalty": 0    # Encourage new topics
    }
    
    # Make API call
    response = requests.post(api_url, headers=headers, json=payload, timeout=120)
    
    # Parse response
    result = response.json()
    return result["choices"][0]["message"]["content"]
```


### 2. Key Advanced Features:
- **System Prompts**: Set the AI's behavior and role
- **Variable Substitution**: Dynamic content with `.format(**variables)`
- **Multi-message Conversations**: System + User + Assistant history
- **Advanced Parameters**: 
  - `temperature`: Creativity level (0.0-1.0)
  - `top_p`: Alternative to temperature (0.0-1.0)
  - `max_tokens`: Response length limit
  - `frequency_penalty`: Reduce repetitive words
  - `presence_penalty`: Encourage topic diversity

### 3. Usage Examples:
```python
# Example 1: Simple variable substitution
user_template = "Create a schedule for {date} with tasks: {tasks}"
variables = {"date": "2025-10-28", "tasks": "meeting, study, exercise"}

# Example 2: Complex system prompt
system_prompt = """You are an expert scheduler. Always respond in JSON format 
with fields: date, start_time, end_time, activity, location, type."""

response = advanced_llm_call(user_template, variables, system_prompt)
```

## Advanced LLM Call Template with Variables 🚀

Here's a more sophisticated template that includes system prompts, user prompts with variables, and advanced configuration options:

### 1. Advanced Structure with System & User Prompts
```python
def advanced_llm_call(user_input, system_prompt, api_endpoint, api_key, variables=None, system_prompt=None):
    # Configuration
    api_url = f"{openai_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
    
    # Default system prompt if none provided
    if system_prompt is None:
        system_prompt = "You are a helpful AI assistant."
    
    # Format user input with variables if provided
    if variables:
        formatted_user_input = user_input.format(**variables)
    else:
        formatted_user_input = user_input
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key  # Azure style
        # "Authorization": f"Bearer {api_key}"  # OpenAI style
    }
    
    # Advanced payload with system and user messages
    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": formatted_user_input}
        ],
        "temperature": 0.1,      # Low for consistent results
        "max_tokens": 4000,      # Longer responses
        "top_p": 0.9,           # Nucleus sampling
        "frequency_penalty": 0,  # Reduce repetition
        "presence_penalty": 0    # Encourage new topics
    }
    
    # Make API call
    response = requests.post(api_url, headers=headers, json=payload, timeout=120)
    
    # Parse response
    result = response.json()
    return result["choices"][0]["message"]["content"]
```


### 2. Key Advanced Features:
- **System Prompts**: Set the AI's behavior and role
- **Variable Substitution**: Dynamic content with `.format(**variables)`
- **Multi-message Conversations**: System + User + Assistant history
- **Advanced Parameters**: 
  - `temperature`: Creativity level (0.0-1.0)
  - `top_p`: Alternative to temperature (0.0-1.0)
  - `max_tokens`: Response length limit
  - `frequency_penalty`: Reduce repetitive words
  - `presence_penalty`: Encourage topic diversity

### 3. Usage Examples:
```python
# Example 1: Simple variable substitution
user_template = "Create a schedule for {date} with tasks: {tasks}"
variables = {"date": "2025-10-28", "tasks": "meeting, study, exercise"}

# Example 2: Complex system prompt
system_prompt = """You are an expert scheduler. Always respond in JSON format 
with fields: date, start_time, end_time, activity, location, type."""

response = advanced_llm_call(user_template, variables, system_prompt)
```

### 📝 Task 2.2: use the prompt & the system prompt to create the schedule 

Make use of fomatted user prompt & system prompt 

In [2]:
def llm_call_with_prompt_and_variables(user_prompt, system_prompt, api_key, openai_endpoint, openai_deployment_name, openai_version_name, date, tasks):
    """
    Simple function to call an LLM with a prompt
    
    Args:
        prompt (str): The prompt to send to the LLM
        max_tokens (int): Maximum tokens in response
    
    Returns:
        str: The LLM's response
    """
    

    # Azure OpenAI API call
    api_url = f"{openai_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
        
    headers = {
            "Content-Type": "application/json",
            "api-key": api_key
        }
    
    formatted_user_prompt = user_prompt.format(date=date, tasks=tasks)
    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": formatted_user_prompt
                    }
                ]
            }
        ],
        "temperature": 0.1,
        "max_tokens": 4000
                }

    # Make the API call
    response = requests.post(  #POST REQUEST
        api_url,
        headers=headers,
        json=payload,
        timeout=120  # Longer timeout for image processing
        )
    # Parse the response
    response_data = response.json()
    content = response_data["choices"][0]["message"]["content"]
    return content

In [6]:
# 🧪 Test the prompt generation
sample_tasks = ["Machine Learning lecture", "Python programming lab", "Team project meeting", "Study time"]
json_schedule = llm_call_with_prompt_and_variables(schedule_user_prompt, schedule_system_prompt, azure_openai_api_key, azure_openai_endpoint, openai_deployment_name, openai_version_name, "2025-10-28", sample_tasks)

print(json_schedule)

```json
{
    "2025-10-28": {
        "schedule": [
            {
                "start_time": "08:00",
                "end_time": "09:30",
                "task": "Machine Learning lecture",
                "location": "Lecture Hall 3",
                "type": "Study"
            },
            {
                "start_time": "09:30",
                "end_time": "10:00",
                "task": "Morning Break",
                "location": "Cafeteria",
                "type": "Break"
            },
            {
                "start_time": "10:00",
                "end_time": "12:00",
                "task": "Python programming lab",
                "location": "Computer Lab 2",
                "type": "Study"
            },
            {
                "start_time": "12:00",
                "end_time": "13:00",
                "task": "Lunch Break",
                "location": "Cafeteria",
                "type": "Break"
            },
            {
                "start_time": 

### 📝 Task 2.3: Extract Events from Image using GPT-4o Vision 📸

Now let's use GPT-4o's vision capabilities to extract events from an image (download.jpeg) and filter them for today's date. This demonstrates multimodal AI capabilities!

In [9]:
# 📸 Image Vision System and User Prompts for Event Extraction
import base64
from datetime import datetime, date



# System prompt for image-based event extraction
image_vision_system_prompt = """
You are an expert academic schedule extraction assistant with vision capabilities.
Your goal is to analyze images of class timetables and extract today's classes.

Current date: {today_date}
Current weekday: {today_weekday}
Current ISO week number: {today_week_number}

EXTRACTION RULES:
1. Carefully examine the image for any weekly schedule information.
2. Identify the column corresponding to today's weekday ({today_weekday}).
3. Extract all class entries for that day, including start and end times, subject names, and notes (e.g., 'even weeks', 'odd weeks', 'week 1-6', etc.).
4. If the class applies only to certain weeks, check whether {today_week_number} fits the condition:
   - If the note says "even weeks" and {today_week_number} is even → include it.
   - If the note says "odd weeks" and {today_week_number} is odd → include it.
   - If the note says "week X–Y" → include only if {today_week_number} is within that range.
5. Ignore all other days and weeks not matching today's conditions.
6. If no classes apply today, return an empty schedule.

RESPONSE FORMAT:
Return the extracted events in this JSON structure:
{{
    "{today_date}": {{
        "schedule": [
            {{
                "start_time": "HH:MM",
                "end_time": "HH:MM",
                "activity": "Course name from image",
                "location": "Location if visible, else null",
                "type": "Categorize as 'Study'"
            }}
        ]
    }}
}}

ADDITIONAL RULES:
- Use 24-hour time format (HH:MM).
- Merge cells or notes (like "PC-oefeningen" or "practicum") into the same activity field if relevant.
- Translate Dutch day or week labels (e.g. "even weken", "oneven weken") into English meaning when applying logic.
- If no classes for today are found, return:
  {{"{today_date}": {{"schedule": []}}}}
- Return only valid JSON output — no extra explanations, no markdown.
"""


# User prompt for image analysis
image_vision_user_prompt = """
Please analyze the attached image of a weekly class timetable and extract all lessons or events that occur today ({today_date}),
which is a {today_weekday} in ISO week {today_week_number}.

Instructions:
- Identify the column corresponding to {today_weekday}.
- Extract all lessons listed for that day, including:
  - Subject or course name
  - Start and end time
  - Notes such as "even weeks", "odd weeks", or "week ranges"
  - Type (e.g., theory, exercises, practicum)
- If a note restricts the class to certain weeks, only include it if {today_week_number} matches the condition.
  - Example: include "even weeks" only if this is an even week.
- If no classes apply today, return an empty schedule.

Output requirements:
- Use the JSON format specified in the system prompt
- Use 24-hour time format
- Do not add explanations or text outside the JSON output
"""


print("✅ Image vision prompts created!")
print("🎯 Ready to process download.jpeg for today's events!")

✅ Image vision prompts created!
🎯 Ready to process download.jpeg for today's events!


## Advanced Vision Template with Image Processing 📸

Here's a template for working with GPT-4o vision capabilities to process images:

### 1. Vision API Structure with Base64 Encoding
```python
def extract_events_from_image(image_path = None, user_input, system_prompt=None, variables=None, api_endpoint = None, api_key = None, openai_deployment_name= None, openai_version_name = None):
    try:
        # Step 1: Read and encode image to base64
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')
        
        # Step 2: Create API endpoint for vision
        api_url = f"{api_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
        
        # Step 3: Set up headers (same as text API)
        headers = {
            "Content-Type": "application/json",
            "api-key": azure_openai_api_key
        }
        
        # Step 4: Format prompts with variables
        formatted_system_prompt = system_prompt.format(
            **variables
        )
        formatted_user_prompt = user_prompt.format(
            **variables
        )
        
        # Step 5: Create payload with image and text
        payload = {
            "messages": [
                {"role": "system", "content": formatted_system_prompt},
                {
                    "role": "user", 
                    "content": [
                        {
                            "type": "text",
                            "text": formatted_user_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_data}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.1,
            "max_tokens": 4000
        }
        
        # Step 6: Make API call with longer timeout
        response = requests.post(api_url, headers=headers, json=payload, timeout=120)
        
        # Step 7: Parse and return response
        response_data = response.json()
        return response_data["choices"][0]["message"]["content"]
        
    except Exception as e:
        return f'Error: {str(e)}'
```

### 2. Key Vision API Features:
- **Base64 Encoding**: Images must be converted to base64 strings
- **Multi-modal Content**: User message contains both text and image_url
- **Image URL Format**: `data:image/jpeg;base64,{base64_string}`
- **Longer Timeout**: Vision processing takes more time (120 seconds recommended)
- **Variable Substitution**: Format prompts with dynamic date/time information

### 3. Content Structure for Vision:
```python
# For vision API, user content is an array with different types:
"content": [
    {
        "type": "text",
        "text": "Your text prompt here"
    },
    {
        "type": "image_url", 
        "image_url": {
            "url": "data:image/jpeg;base64,{your_base64_image}"
        }
    }
]
```

### 4. Common Vision Use Cases:
- **Schedule Extraction**: Read timetables and extract events
- **Document Analysis**: Parse forms, receipts, or structured documents  
- **Academic Calendar Processing**: Extract class schedules with week-specific rules
- **Visual Data Entry**: Convert image information to structured JSON

This template enables powerful multimodal AI capabilities!

In [10]:
def extract_events_from_image(image_path="lessenrooster.png", today_date = None, today_weekday = None, today_week_number = None):
    """
    Extract events from an image using GPT-4o vision capabilities
    
    Args:
        image_path (str): Path to the image file
        
    Returns:
        str: JSON response with extracted events for today's date
    """
    
    try:
        # Read and encode the image
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')
        
        # Azure OpenAI API call with vision
        api_url = f"{azure_openai_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
            
        headers = {
                "Content-Type": "application/json",
                "api-key": azure_openai_api_key
            }
        formatted_system_prompt = image_vision_system_prompt.format(today_date=today_date, today_weekday=today_weekday, today_week_number=today_week_number)
        formatted_user_prompt = image_vision_user_prompt.format(today_date=today_date, today_weekday=today_weekday, today_week_number=today_week_number)
        # Payload with image and text
        payload = {
            "messages": [
                {"role": "system", "content": formatted_system_prompt},
                {
                    "role": "user", 
                    "content": [
                        {
                            "type": "text",
                            "text": formatted_user_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_data}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.1,
            "max_tokens": 4000
        }

        # Make the API call
        response = requests.post(
            api_url,
            headers=headers,
            json=payload,
            timeout=120  # Longer timeout for image processing
        )
        
        # Parse the response
        response_data = response.json()
        content = response_data["choices"][0]["message"]["content"]
        return content
        
    except FileNotFoundError:
        return f'Error: Image file "{image_path}" not found. Please ensure download.jpeg exists in the current directory.'
    except Exception as e:
        return f'Error processing image: {str(e)}'

# Test the image extraction function
print("🔍 Extracting events from lessenrooster.png")
try:
    # Get today's date for filtering events
    today = date.today()
    today_date = today.isoformat()

    today_weekday = today.strftime("%A")
    today_week_number = today.isocalendar()[1]

    image_events_response = extract_events_from_image(today_date=today_date, today_weekday=today_weekday, today_week_number=today_week_number)
    print("📸 GPT-4o Vision Response:")
    print(image_events_response)
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Make sure 'download.jpeg' exists in your project directory")

🔍 Extracting events from lessenrooster.png
📸 GPT-4o Vision Response:
{
    "2025-11-03": {
        "schedule": [
            {
                "start_time": "08:15",
                "end_time": "10:15",
                "activity": "Dierkunde theorie",
                "location": null,
                "type": "Study"
            },
            {
                "start_time": "10:30",
                "end_time": "12:30",
                "activity": "Algemene chemie I theorie",
                "location": null,
                "type": "Study"
            }
        ]
    }
}


Extract the json. Now it remains text, but it should be a json format. 

In [11]:
def extract_json_schedule(llm_response):
    """
    Extract JSON schedule from LLM response string.
    
    Args:
        llm_response (str): The raw response from the LLM.
    
    Returns:
        dict: Parsed JSON schedule.
    """
    try:
        # Use regex to find the JSON part of the response
        json_match = re.search(r'\{.*\}', llm_response, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            schedule = json.loads(json_str)
            return schedule
        else:
            raise ValueError("No JSON found in LLM response.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")
    
# Extract and print the structured schedule
structured_schedule = extract_json_schedule(image_events_response)
print("📅 Structured Schedule:")
print(structured_schedule)

📅 Structured Schedule:
{'2025-11-03': {'schedule': [{'start_time': '08:15', 'end_time': '10:15', 'activity': 'Dierkunde theorie', 'location': None, 'type': 'Study'}, {'start_time': '10:30', 'end_time': '12:30', 'activity': 'Algemene chemie I theorie', 'location': None, 'type': 'Study'}]}}


## Exercise 3: Parse LLM Response 🔍

Great! Now we have a schedule from the LLM. But it's in text format - we need to parse it into structured data that we can use with the Google Calendar API.

### 📝 Task 3.1: Extract the first entry

Let's create a function to get the first entry and add the date in it as well. 

In [13]:
def extract_first_entry(parsed_schedule, date):
    """
    Extract the first schedule entry and format it for calendar creation
    
    Args:
        parsed_schedule (dict): Parsed schedule data
        date (str): The date for the schedule
        
    Returns:
        dict: First calendar entry or None
    """

    first_entry = parsed_schedule[date]["schedule"][0].copy()  # Create a copy to avoid modifying original
    first_entry['date'] = date  # Add the date to the entry
    return first_entry
first_entry = extract_first_entry(structured_schedule, "2025-11-03")
print(first_entry)

{'start_time': '08:15', 'end_time': '10:15', 'activity': 'Dierkunde theorie', 'location': None, 'type': 'Study', 'date': '2025-11-03'}


## Google Calendar API Setup Instructions 🔧

Before we can schedule events, we need to set up Google Calendar API access. Follow these steps carefully:

### Step 1: Create a Google Cloud Project

1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Click "Create Project" or select an existing project
3. Give your project a name (e.g., "Calendar Integration Project")

### Step 2: Enable Calendar API

1. In the Google Cloud Console, go to "APIs & Services" > "Library"
2. Search for "Google Calendar API"
3. Click on it and press "Enable"

### Step 3: Create Service Account Credentials

1. Go to "APIs & Services" > "Credentials"
2. Click "Create Credentials" > "Service Account"
3. Give it a name (e.g., "calendar-service-account")
4. Click "Create and Continue"
5. Skip role assignment (click "Continue")
6. Click "Done"

### Step 4: Generate and Download Key

1. Click on your newly created service account
2. Go to the "Keys" tab
3. Click "Add Key" > "Create New Key"
4. Choose "JSON" format
5. Save your clientid & clientsecret in the config file of earlier. 

### Step 5: Share Your Calendar

1. Open Google Calendar in your browser
2. Find your calendar in the left sidebar
3. Click the three dots next to it > "Settings and sharing"
4. In "Share with specific people", add your service account email (found in the JSON file)
5. Give it "Make changes to events" permission

### ⚠️ Security Note
Never commit your `credentials.json` file to version control! Add it to your `.gitignore`.

## Exercise 4: Google Calendar Authentication 🔐

Now let's implement Google Calendar API authentication. Make sure you've completed the setup steps above!

In [10]:
# Get Google API credentials from environment variables
import os
Client_id = os.getenv("GOOGLE_CLIENT_ID")
client_secret = os.getenv("GOOGLE_CLIENT_SECRET")

# If modifying these scopes, delete the file token.pickle.

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(
                {
                    "installed": {
                        "client_id": Client_id,
                        "client_secret": client_secret,
                        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                        "token_uri": "https://oauth2.googleapis.com/token"
                    }
                },
                SCOPES
            )
            creds = flow.run_local_server(port=0)  # automatically opens browser and handles redirect

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    return service

In [11]:
get_calendar_service()

## Exercise 5: Create Calendar Event 📅

Great! Now we can create our calendar event function. This will take our parsed schedule entry and create an actual Google Calendar event.
Looking at the documentation of the google api, the event of a calendar has a specific structure. 

### 📝 Task 5.1: Build the Event Creation Function

In [12]:
def create_calendar_event(event_data):
    """
    Create a Google Calendar event from parsed schedule data
    
    Args:
        event_data (dict): Event information with fields like:
            - date: Event date (YYYY-MM-DD)
            - start_time: Start time (HH:MM)
            - end_time: End time (HH:MM)
            - activity: Event title/name
            - location: Event location (optional)
            - type: Event type (optional)
        
    Returns:
        dict: Result of event creation
    """
    
    try:
        # Get calendar service
        service = get_calendar_service()
        
        if not service:
            return {"success": False, "error": "No calendar service available"}
        
        # Parse date and time
        event_date = event_data["date"]
        start_time = event_data["start_time"]
        end_time = event_data["end_time"]
        
        # Create datetime strings (ISO format for Google Calendar)
        start_datetime = f"{event_date}T{start_time}:00"
        end_datetime = f"{event_date}T{end_time}:00"
        
        # Get event title - try different possible field names
        event_title = event_data.get("activity") or event_data.get("task") or "Scheduled Event"
        event_type = event_data.get("type", "")
        
        # Build the event structure
        event = {
            'summary': f"{event_title}" + (f" - {event_type}" if event_type else ""),
            'location': event_data.get("location", ""),
            'description': event_data.get("description", f"Auto-scheduled: {event_title}"),
            'start': {
                'dateTime': start_datetime,
                'timeZone': 'Europe/Brussels',  # Adjust for your timezone
            },
            'end': {
                'dateTime': end_datetime,
                'timeZone': 'Europe/Brussels',
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 15},
                    {'method': 'email', 'minutes': 60},
                ],
            },
        }
        
        # Create the event
        print(f"📅 Creating event: {event_title}")
        print(f"🕒 Time: {start_time} - {end_time} on {event_date}")
        
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        
        return {
            "success": True,
            "event_id": created_event.get('id'),
            "event_link": created_event.get('htmlLink'),
            "message": f"Successfully created event: {event_title}",
            "event_details": {
                "title": event_title,
                "date": event_date,
                "start_time": start_time,
                "end_time": end_time,
                "location": event_data.get("location", "")
            }
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": f"Failed to create event: {str(e)}"
        }

# 🧪 Test function (we'll create the actual event in the next section)
print("✅ Calendar event creation function ready!")
print("🎯 We'll use this to schedule our first LLM-generated event next!")

✅ Calendar event creation function ready!
🎯 We'll use this to schedule our first LLM-generated event next!


## Exercise 6: Complete Integration Test 🎯

This is the exciting part! Let's bring everything together and create a complete MCP function calling workflow that:

1. ✅ Uses LLM to generate a schedule
2. ✅ Parses the response 
3. ✅ Extracts the first entry
4. ✅ Creates a Google Calendar event

### 📝 Task 6.1: Create Your First Automated Calendar Event

**Important**: This will create a REAL event in your Google Calendar!

In [13]:
first_entry = {'start_time': '08:15', 'end_time': '10:15', 'activity': 'Plantkunde theorie', 'location': None, 'type': 'Study', 'date': '2025-10-28'}
event_result = create_calendar_event(first_entry)
print(event_result)

📅 Creating event: Plantkunde theorie
🕒 Time: 08:15 - 10:15 on 2025-10-28
{'success': True, 'event_id': 'mih35b37kbmlc766mb06vcu3qo', 'event_link': 'https://www.google.com/calendar/event?eid=bWloMzViMzdrYm1sYzc2Nm1iMDZ2Y3UzcW8ganVsaWV2YW5hY2tlcmUxNEBt', 'message': 'Successfully created event: Plantkunde theorie', 'event_details': {'title': 'Plantkunde theorie', 'date': '2025-10-28', 'start_time': '08:15', 'end_time': '10:15', 'location': None}}


## Exercise 7: Let's make an MCP setup
A more agentic setup is more about letting the LLM choose the right tools to work with. 
you can define the funtions as tools and then ask the LLM to schedule the event itself --> ideal for change management or checking if there is an overlapping event. 
1. Define the function of the creation of an event based on an LLM 
2. Define the functions of the google api: get_calendar_service and create_calendar_event 

In [74]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "llm_call_with_prompt_and_variables",
            "description": "Generate a schedule using LLM for a specific date and tasks",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string", 
                        "description": "The date for the schedule in YYYY-MM-DD format"
                    },
                    "tasks": {
                        "type": "string", 
                        "description": "A description of tasks to schedule (can be a list or comma-separated string)"
                    }
                },
                "required": ["date", "tasks"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_json_schedule",
            "description": "Parse JSON schedule from LLM text response",
            "parameters": {
                "type": "object",
                "properties": {
                    "llm_response": {
                        "type": "string", 
                        "description": "The raw text response from the LLM containing JSON"
                    }
                },
                "required": ["llm_response"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_first_entry",
            "description": "Get the first schedule entry from a parsed schedule",
            "parameters": {
                "type": "object",
                "properties": {
                    "parsed_schedule": {
                        "type": "object", 
                        "description": "The parsed schedule data as JSON"
                    },
                    "date": {
                        "type": "string", 
                        "description": "The date to extract the first entry from"
                    }
                },
                "required": ["parsed_schedule", "date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "Create a Google Calendar event from event data",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_data": {
                        "type": "object",
                        "description": "Event data containing date, start_time, end_time, activity/task, location, type",
                        "properties": {
                            "date": {"type": "string", "description": "Event date in YYYY-MM-DD format"},
                            "start_time": {"type": "string", "description": "Start time in HH:MM format"},
                            "end_time": {"type": "string", "description": "End time in HH:MM format"},
                            "activity": {"type": "string", "description": "Event title/activity name"},
                            "location": {"type": "string", "description": "Event location (optional)"},
                            "type": {"type": "string", "description": "Event type (optional)"}
                        },
                        "required": ["date", "start_time", "end_time", "activity"]
                    }
                },
                "required": ["event_data"]
            }
        }
    }
]

In [75]:
# Azure OpenAI setup
api_url = f"{azure_openai_endpoint.rstrip('/')}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_version_name}"
    
headers = {
    "Content-Type": "application/json",
    "api-key": azure_openai_api_key
}
    
# Updated system message with clearer instructions
messages = [
    {
        "role": "system", 
        "content": """You are an autonomous scheduling assistant that can plan tasks and add events to Google Calendar.

Your workflow should be:
1. First, use llm_call_with_prompt_and_variables to generate a schedule for the requested date and tasks
2. Then, use extract_json_schedule to parse the LLM response into structured data  
3. Next, use extract_first_entry to get the first event from the schedule
4. Finally, use create_calendar_event to add the event to Google Calendar

Always follow this sequence to successfully create calendar events."""
    },
    {
        "role": "user", 
        "content": "Create a schedule for 2025-10-28 with these tasks: Machine Learning lecture, Python programming lab, Team project meeting, Study time. Then add the first event to my Google Calendar."
    }
]

max_iterations = 10

for i in range(max_iterations):
    payload = {
        "messages": messages,
        "tools": tools,
        "tool_choice": "auto",
        "temperature": 0.1,
        "max_tokens": 1500
    }

    response = requests.post(api_url, headers=headers, json=payload, timeout=60)
    if response.status_code != 200:
        print(f"❌ API Error {response.status_code}: {response.text}")
        break

    result = response.json()
    assistant_msg = result["choices"][0]["message"]
    
    # Add assistant message to conversation
    messages.append(assistant_msg)

    # --- Tool calls must come from assistant ---
    if "tool_calls" in assistant_msg:
        for call in assistant_msg["tool_calls"]:
            fn_name = call["function"]["name"]
            args = json.loads(call["function"]["arguments"])

            print(f"🧠 Iteration {i+1}: Assistant called {fn_name}")
            print(f"   Args: {args}")

            try:
                # Execute Python function
                fn = globals()[fn_name]
                output = fn(**args)
                
                print(f"   ✅ Result: {output if isinstance(output, (str, int, bool)) else 'Success'}")

                # Append tool result properly
                messages.append({
                    "role": "tool",
                    "tool_call_id": call["id"],  # MUST match assistant's tool_call ID
                    "content": json.dumps(output) if not isinstance(output, str) else output
                })
                
            except Exception as e:
                print(f"   ❌ Error: {str(e)}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": call["id"],
                    "content": f"Error: {str(e)}"
                })

    # --- Assistant finished reasoning, no more tool calls ---
    else:
        print(f"\n✅ Final Assistant Message (Iteration {i+1}):")
        print(assistant_msg.get("content", ""))
        break

print("\n🎉 MCP Function Calling Complete!")

🧠 Iteration 1: Assistant called llm_call_with_prompt_and_variables
   Args: {'date': '2025-10-28', 'tasks': 'Machine Learning lecture, Python programming lab, Team project meeting, Study time'}
   ✅ Result: {
    "2025-10-28": {
        "schedule": [
            {
                "start_time": "08:00",
                "end_time": "09:30",
                "task": "Machine Learning lecture",
                "location": "Lecture Hall 3",
                "type": "Study"
            },
            {
                "start_time": "09:30",
                "end_time": "10:00",
                "task": "Morning Break",
                "location": "Cafeteria",
                "type": "Break"
            },
            {
                "start_time": "10:00",
                "end_time": "12:00",
                "task": "Python programming lab",
                "location": "Computer Lab 2",
                "type": "Study"
            },
            {
                "start_time": "12:00",
         